In [ ]:
#use CPU for IQA as CUDA in GPU is not compatible with the python version
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# Import library

In [ ]:
import imquality.brisque as brisque
import PIL.Image
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import ImageFile
from tensorflow.keras.preprocessing import image
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as PSNR

# Load Data, preprocess

In [ ]:
disease_img = {'Atelectasis': [],
                 'Cardiomegaly': [],
                 'Effusion': [],
                 'Infiltration': [],
                 'Mass': [],
                 'Nodule': [],
                 'Pneumonia': [],
                 'Pneumothorax': [],
                 'Consolidation': [],
                 'Edema': [],
                 'Emphysema': [],
                 'Fibrosis': [],
                 'Pleural_Thickening': [],
                 'Hernia': [],
                 'No Finding':[]}

In [ ]:
disease_class = {'Atelectasis': 1,
                 'Cardiomegaly': 2,
                 'Effusion': 3,
                 'Infiltration': 4,
                 'Mass': 5,
                 'Nodule': 6,
                 'Pneumonia': 7,
                 'Pneumothorax': 8,
                 'Consolidation': 9,
                 'Edema': 10,
                 'Emphysema': 11,
                 'Fibrosis': 12,
                 'Pleural_Thickening': 13,
                 'Hernia': 14,
                 'No Finding': 15}

disease_rev = {v: k for k, v in disease_class.items()}

In [ ]:
data_ref = pd.read_csv("/media/ntu/volume1/home/s123md305_01/Documents/CXR8/Data_Entry_2017_v2020.csv")
pd.options.mode.chained_assignment = None        #ignore the SettingWithCopyWarning
img_names=[]
img_names2=[]

#/media/ntu/volume1/home/s123md305_01/Documents/Generated/reconstructed_labels.csv
#/media/ntu/volume1/home/s123md305_01/Documents/CXR8/Data_Entry_2017_v2020.csv
for i in tqdm(range(len(data_ref))):
    #print(i)
    if "|" not in data_ref['Finding Labels'][i]:
        disease_img[data_ref['Finding Labels'][i]].append(data_ref['Image Index'][i])
        img_names.append(data_ref['Image Index'][i])

data_ref_2 = pd.read_csv("/media/ntu/volume1/home/s123md305_01/Documents/Generated/reconstructed_labels.csv")  # Update the path to the second dataset

# Initialize dictionary for the second dataset
disease_img_2 = {disease: [] for disease in disease_class.keys()}

# Populate the dictionary with image names from the second dataset
for i in tqdm(range(len(data_ref_2))):
    if "|" not in data_ref_2['Finding Labels'][i]:
        disease_img_2[data_ref_2['Finding Labels'][i]].append(data_ref_2['Image Index'][i])
        img_names2.append(data_ref_2['Image Index'][i])

# If you need a simplified reference for the second dataset as well


# PSNR

In [ ]:
import warnings

    
warnings.simplefilter(action='ignore', category=FutureWarning)

quality = pd.DataFrame(columns=['styleGAN VS Original'])

# Assuming you meant to collect all the original and GAN images in lists
ori_images = []
gan_images = []

for img_name in tqdm(img_names):
    img_path = '/media/ntu/volume1/home/s123md305_01/Documents/CombinedResized/Resized112/' + img_name
    img = image.load_img(img_path, target_size=(112, 112, 3))
    ori_images.append(image.img_to_array(img))

for img_name in tqdm(img_names2):
    img_path = '/media/ntu/volume1/home/s123md305_01/Documents/Generated/ComGenerated112/' + img_name
    img = image.load_img(img_path, target_size=(112, 112, 3))
    gan_images.append(image.img_to_array(img))

gan_scores = []

for ori, gan in zip(ori_images, gan_images):
    gan_score = PSNR(ori, gan, data_range=255)
    gan_scores.append({'styleGAN VS Original': gan_score})

# Convert the list of dictionaries to a DataFrame before concatenating
gan_scores_df = pd.DataFrame(gan_scores)
quality = pd.concat([quality, gan_scores_df], ignore_index=True)

quality.to_csv('/media/ntu/volume1/home/s123md305_01/Documents/Results/IQA_PSNR.csv')

In [ ]:
bin_values = np.arange(start=0, stop=25, step=1)

In [ ]:
noise = quality['styleGAN VS Original']
noise.plot(kind = 'hist', bins=bin_values, figsize=[15,10],alpha=.4, legend = 'Ture')
#gan = quality['GAN']
#gan.plot(kind = 'hist', bins=bin_values, figsize=[15,10],alpha=.4, legend = 'Ture')

# BRISQUE

In [ ]:
# BRISQUE (DCGAN, Original, Noise)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
quality = pd.DataFrame(columns = ['Img_Name','Original', 'styleGAN'])

for i in tqdm(range(len(img_names))):
    name = img_names[i]
    ori = PIL.Image.open('/media/ntu/volume1/home/s123md305_01/Documents/CombinedResized/Resized112/'+name)
    ori_score = brisque.score(ori)
    
    gan = PIL.Image.open('/media/ntu/volume1/home/s123md305_01/Documents/Generated/IndexMatched112/ComGenerated112/'+name)
    gan_score = brisque.score(gan)

    quality = quality.append({'Img_Name': name, 'Original': ori_score, 'DCGAN': gan_score}, ignore_index=True)

quality.to_csv('IQA_BRISQUE.csv')

In [ ]:
bin_values = np.arange(start=-50, stop=200, step=10)

original = scores['Original_BRISQUE']
original.plot(kind = 'hist', bins=bin_values, figsize=[15,10], legend = 'Ture')
noise = scores['Stylegan_BRISQUE']
noise.plot(kind = 'hist', bins=bin_values, figsize=[15,10],alpha=.4, legend = 'Ture')


In [ ]:
import pandas as pd

# Create a simple DataFrame
df = pd.DataFrame({'A': [1, 2, 3]})
# Try appending a new row
df = df.append({'A': 4}, ignore_index=True)
print(df)


In [ ]:
import pandas as pd
import os
import shutil

# Load the CSV files
data_ref = pd.read_csv("/media/ntu/volume1/home/s123md305_01/Documents/CXR8/Data_Entry_2017_v2020.csv")
data_ref_2 = pd.read_csv("/media/ntu/volume1/home/s123md305_01/Documents/Generated/reconstructed_labels.csv")

# Directory where images will be renamed
image_dir = "/media/ntu/volume1/home/s123md305_01/Documents/Generated/IndexMatched224/ComGenerated224"

# Create a mapping from 'Finding Labels' to a set of 'Image Index' from the first CSV
label_to_images_map = {}
for index, row in data_ref.iterrows():
    label = row['Finding Labels']
    if label not in label_to_images_map:
        label_to_images_map[label] = set()
    label_to_images_map[label].add(row['Image Index'])

# Iterate over the second DataFrame and rename images in the folder
for index, row in data_ref_2.iterrows():
    label_2 = row['Finding Labels']
    image_name_2 = row['Image Index']

    if label_2 in label_to_images_map:
        # Get a non-used image name from the set, if available
        if label_to_images_map[label_2]:
            new_image_name = label_to_images_map[label_2].pop()  # Remove an item to prevent reuse

            original_path = os.path.join(image_dir, image_name_2)
            new_path = os.path.join(image_dir, new_image_name)

            if os.path.exists(original_path):
                os.rename(original_path, new_path)
                print(f"Renamed {original_path} to {new_path}")
            else:
                print(f"File {original_path} does not exist.")
        else:
            print(f"No available unique names left for label {label_2}.")
    else:
        print(f"Label {label_2} from second CSV not found in first CSV.")


In [ ]:
import skimage as sk
sk.__version__

In [ ]:
import warnings
import pandas as pd
import PIL.Image
import imquality.brisque as brisque
from tqdm import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)

# Initialize an empty list to store the scores
scores = []

# Loop through the image names
for i in tqdm(range(len(img_names))):
    name = img_names[i]

    # Load the original image and calculate its BRISQUE score
    ori_path = '/media/ntu/volume1/home/s123md305_01/Documents/CombinedResized/Resized112/' + name
    ori = PIL.Image.open(ori_path)
    ori_score = brisque.score(ori)

    # Load the StyleGAN image and calculate its BRISQUE score
    gan_path = '/media/ntu/volume1/home/s123md305_01/Documents/Generated/IndexMatched112/ComGenerated112/' + name
    gan = PIL.Image.open(gan_path)
    gan_score = brisque.score(gan)

    # Append the scores and image name to the list
    scores.append({'Img_Name': name, 'Original_BRISQUE': ori_score, 'StyleGAN_BRISQUE': gan_score})

# Convert the list of dictionaries to a DataFrame
scores_df = pd.DataFrame(scores)

# Save the resulting DataFrame to a CSV file
scores_df.to_csv('/media/ntu/volume1/home/s123md305_01/Documents/Results/IQA_BRISQUE.csv')


In [ ]:
bin_values = np.arange(start=-50, stop=200, step=10)

# Plotting
import matplotlib.pyplot as plt

plt.figure(figsize=[15,10])
plt.hist(scores_df['Original_BRISQUE'], bins=bin_values, alpha=0.7, label='Original')
plt.hist(scores_df['StyleGAN_BRISQUE'], bins=bin_values, alpha=0.7, label='StyleGAN')
plt.title('Histogram of BRISQUE Scores')
plt.xlabel('BRISQUE Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming scores_df is defined and contains your data

# Define bin values for the histogram
bin_values = np.arange(start=-50, stop=200, step=10)

# Plotting
plt.figure(figsize=[15, 10])
plt.hist(scores_df['Original_BRISQUE'], bins=bin_values, alpha=0.7, label='Original')
plt.hist(scores_df['StyleGAN_BRISQUE'], bins=bin_values, alpha=0.7, label='StyleGAN')
plt.title('Histogram of BRISQUE Scores')
plt.xlabel('BRISQUE Score')
plt.ylabel('Frequency')
plt.legend()
plt.xlim(-50, 100)  # Limiting the x-axis to -50 to 100
plt.show()


In [ ]:
import warnings
import pandas as pd
import PIL.Image
import imquality.brisque as brisque
from tqdm import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)

# Initialize an empty list to store the scores
scores2 = []

# Loop through the image names
for i in tqdm(range(len(img_names))):
    name = img_names[i]

    # Load the original image and calculate its BRISQUE score
    ori_path = '/media/ntu/volume1/home/s123md305_01/Documents/CombinedResized/Resized224/' + name
    ori = PIL.Image.open(ori_path)
    ori_score = brisque.score(ori)

    # Load the StyleGAN image and calculate its BRISQUE score
    gan_path = '/media/ntu/volume1/home/s123md305_01/Documents/Generated/IndexMatched224/ComGenerated224/' + name
    gan = PIL.Image.open(gan_path)
    gan_score = brisque.score(gan)

    # Append the scores and image name to the list
    scores2.append({'Img_Name': name, 'Original_BRISQUE': ori_score, 'StyleGAN_BRISQUE': gan_score})

# Convert the list of dictionaries to a DataFrame
scores_df2 = pd.DataFrame(scores2)

# Save the resulting DataFrame to a CSV file
scores_df2.to_csv('/media/ntu/volume1/home/s123md305_01/Documents/Results/IQA_BRISQUE2.csv')


In [ ]:
import warnings
import pandas as pd
from PIL import Image
import imquality.brisque as brisque
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

warnings.simplefilter(action='ignore', category=FutureWarning)

# Define a function to compute BRISQUE scores for original and StyleGAN images
def compute_brisque_scores(image_name):
    # Paths to the original and StyleGAN images
    ori_path = f'/media/ntu/volume1/home/s123md305_01/Documents/CombinedResized/Resized224/{image_name}'
    gan_path = f'/media/ntu/volume1/home/s123md305_01/Documents/Generated/IndexMatched224/ComGenerated224/{image_name}'
    
    # Load images
    ori = Image.open(ori_path)
    gan = Image.open(gan_path)
    
    # Calculate BRISQUE scores
    ori_score = brisque.score(ori)
    gan_score = brisque.score(gan)
    
    # Return a dictionary with the scores and image name
    return {'Img_Name': image_name, 'Original_BRISQUE': ori_score, 'StyleGAN_BRISQUE': gan_score}

# Assuming img_names is a list of your image names
# img_names = [...]

# Use ProcessPoolExecutor to parallelize the BRISQUE score computation
with ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(compute_brisque_scores, img_names), total=len(img_names)))

# Convert the results to a DataFrame
scores_df2 = pd.DataFrame(results)

# Save the DataFrame to a CSV file
scores_df2.to_csv('/media/ntu/volume1/home/s123md305_01/Documents/Results/IQA_BRISQUE2.csv', index=False)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming scores_df is defined and contains your data

# Define bin values for the histogram
bin_values = np.arange(start=-50, stop=200, step=10)

# Plotting
plt.figure(figsize=[15, 10])
plt.hist(scores_df2['Original_BRISQUE'], bins=bin_values, alpha=0.7, label='Original')
plt.hist(scores_df2['StyleGAN_BRISQUE'], bins=bin_values, alpha=0.7, label='StyleGAN')
plt.title('Histogram of BRISQUE Scores')
plt.xlabel('BRISQUE Score')
plt.ylabel('Frequency')
plt.legend()
plt.xlim(-50, 100)  # Limiting the x-axis to -50 to 100
plt.show()
